In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [12]:
feed.restrict_dates('20140601')

[]

In [8]:
feed.compute_stop_stats(['20140601', '20140603'])

,stop_id,num_routes,num_trips,start_time,end_time,max_headway,min_headway,mean_headway,date
0,750000,1.0,16.0,07:16:00,22:16:00,60.0,60.0,60.000000,20140601
1,750001,1.0,16.0,07:18:00,22:18:00,60.0,60.0,60.000000,20140601
2,750002,1.0,16.0,07:20:00,22:20:00,60.0,60.0,60.000000,20140601
3,750003,1.0,16.0,07:21:00,22:21:00,60.0,60.0,60.000000,20140601
4,750004,1.0,16.0,07:23:00,22:23:00,60.0,60.0,60.000000,20140601
5,750005,1.0,16.0,07:25:00,22:25:00,60.0,60.0,60.000000,20140601
6,750006,1.0,16.0,07:26:00,22:26:00,60.0,60.0,60.000000,20140601
7,750007,1.0,16.0,07:27:00,22:27:00,60.0,60.0,60.000000,20140601
8,750008,1.0,16.0,07:28:00,22:28:00,60.0,60.0,60.000000,20140601
9,750009,1.0,16.0,07:29:00,22:29:00,60.0,60.0,60.000000,20140601


In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()